# **Canais Quânticos - Exemplo 2: Dois nós conectados por um canal clássico**

Neste notebook praticamos como conectar dois nós por um canal clássico. Usamos um programa quântico para enviar um bit clássico. O bit que estamos enviando ditará como modificar o qubit armazenado no segundo nó.

In [1]:
import netsquid as ns
from netsquid.components.qprocessor import QuantumProcessor, PhysicalInstruction
from netsquid.components.qprogram import QuantumProgram
from netsquid.nodes import Node
from netsquid.nodes.connections import Connection
from netsquid.components.cchannel import ClassicalChannel
from netsquid.components.qchannel import QuantumChannel
import netsquid.components.instructions as instr
from netsquid.protocols.nodeprotocols import NodeProtocol
from netsquid.protocols.protocol import Signals
import pydynaa

<br><br> Começaremos novamente definindo a instruções físicas e inicializando dois processadores quânticos com uma posição de memória cada.

In [2]:
phys_instructions2 = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3),
    PhysicalInstruction(instr.INSTR_H, duration=1),
    PhysicalInstruction(instr.INSTR_X, duration=1),
    PhysicalInstruction(instr.INSTR_MEASURE, duration=7)
]
qproc2a = QuantumProcessor(name="ExampleQProc2a" , num_positions =1, phys_instructions = phys_instructions2)
qproc2b = QuantumProcessor(name="ExampleQProc2b" , num_positions =1, phys_instructions = phys_instructions2)

<br><br> Cada processador será novamente atribuído a um nó. Os dois nós estarão fisicamente separados um do outro e nosso objetivo é enviar um bit clássico de um nó para o outro. Para fazer isso, inicializaremos os dois nós com portas para comunicação clássica de entrada (cin) e saída (cout).

In [3]:
node2a = Node(name="ExampleNode2a", qmemory=qproc2a, port_names=['cin_2a','cout_2a'])
node2b = Node(name="ExampleNode2b", qmemory=qproc2b, port_names=['cin_2b','cout_2b'])

<br><br> Nós também inicializamos um canal clássico que usaremos para conectar aos outros dois nós.

<span style="color: orange"> class netsquid.components.cchannel.ClassicalChannel(name, delay=0, lenght=0, models=None, classical_code=None, transmit_empty_items=False, properties=None, **kwargs) </span>

In [4]:
cchannel2=ClassicalChannel(name="CChannel_2a22b")

<br><br> Nosso objetivo é
- Inicialize um qubit no slot 0 no node2a
- Inicialize um qubit no slot 0 em node2b
- Alterar o estado do qubit no node2a para um estado de superposição
- Medir o qubit
- Enviar o resultado da medição através do canal clássico para node2b
- Se o bit clássico for 1, altere o estado do qubit armazenado no node2b

Para fazer isso, precisamos conectar os nós e o canal. <br><br> 
Vamos fazer isso da seguinte forma: <br><br>

- Conectando a saída do node2a com a entrada do cchannel;
- Conectando a saída do cchannel com a entrada do node2b.

Desta forma podemos enviar um bit clássico de node2a para node2b e usar isso para decidir se o estado do qubit em node2b deve ser modificado

In [5]:
node2a.ports['cout_2a'].connect(cchannel2.ports["send"])
node2b.ports['cin_2b'].connect(cchannel2.ports["recv"])

Primeiro, nós inicializamos um qubit no node2b.

In [6]:
node2b.qmemory.execute_instruction(instr.INSTR_INIT, [0]) # Agendando inicialização do qubit
ns.sim_run() # Executando o próximo evento agendado
ns.sim_time() # Checando o tempo atual

3.0

Em seguida,
- Inicializamos um qubit no slot 0 no node2a;
- Alteramos o estado do qubit para  $|+\rangle = \frac{1}{\sqrt 2}(|0\rangle + |1\rangle)$;
- Medimos o qubit.

Para praticar o uso de programas quânticos, agruparemos todas essas etapas em um programa que executaremos no node2a

In [7]:
class node2aProgram(QuantumProgram):
    default_num_qubits = 1 # Número de qubits
    
    def program(self):
        q0, = self.get_qubit_indices(1) # obtendo os índices para o qubit que estaremos trabalhando
        self.apply(instr.INSTR_INIT, [q0]) # Inicializando o qubit
        
        # Alterando o estado do qubit
        self.apply(instr.INSTR_H, [q0]) # aplicando a porta H no q0
        
        # Agora medindo qubit
        self.apply(instr.INSTR_MEASURE, q0, output_key="m1") # Medindo q0 e armazenando o resultado
        
        yield self.run() # Executando as instruções agendadas acima

<br><br> Executando o programa no node1a

In [8]:
testprog = node2aProgram()
qproc2a.execute_program(testprog)
ns.sim_run()
ns.sim_time()

14.0

Agora, colocamos o resultado da medição na porta de saída do node2a

In [9]:
m1, = testprog.output["m1"]
node2a.ports["cout_2a"].tx_output(m1)
ns.sim_run()
ns.sim_time()

14.0

E por último, usamos a mensagem para decidir se queremos modificar o qubit no node2b.

Se o resultado da medição for
- "1" nos aplicamos à porta H;
- "0" nós aplicamos à porta X.

In [10]:
message = node2b.ports["cin_2b"].rx_input()
if message.items[0] ==1:
    node2b.qmemory.execute_instruction(instr.INSTR_H, [0]) # Aplicando a porta H
    print("Aplicando a porta H")
if message.items[0] ==0:
    node2b.qmemory.execute_instruction(instr.INSTR_X, [0]) # Aplicando a porta X
    print("Aplicando a porta H")
    
ns.sim_run()
ns.sim_time()

Aplicando a porta H


15.0

## Sugestão de Prática

Combine o que praticamos no exemplo 1 e no exemplo 2:
- Crie um processador quântico com duas posições de memória e atribua-o ao nodeA;
- Crie um processador quântico com duas posições de memória e atribua-o ao nodeB;
- Escreva um programa que inicialize os dois qubits no nó A, aplique a porta H ao segundo qubit e então meça o segundo qubit;
- Envie o primeiro qubit para nodeB;
- Envie o resultado da medição para o nodeB;
- Use o resultado da medição para decidir como alterar o estado do qubit no nodeB.